In [86]:
!pip install pinecone

In [87]:
%pip install xmltodict

In [88]:
from pinecone import Pinecone

pc = Pinecone("pcsk_5zSDm7_FiXyttnjXKArVHUKTq2wVnvigTxJD4jQtavV9MJ2kVQbVVjtJuso6pTUkvaJU2W")

# Embed data
data = [
    {"id": "vec1", "text": "Apple is a popular fruit known for its sweetness and crisp texture."},
    {"id": "vec2", "text": "The tech company Apple is known for its innovative products like the iPhone."},
    {"id": "vec3", "text": "Many people enjoy eating apples as a healthy snack."},
    {"id": "vec4", "text": "Apple Inc. has revolutionized the tech industry with its sleek designs and user-friendly interfaces."},
    {"id": "vec5", "text": "An apple a day keeps the doctor away, as the saying goes."},
]

embeddings = pc.inference.embed(
    model="llama-text-embed-v2",
    inputs=[d['text'] for d in data],
    parameters={
        "input_type": "passage"
    }
)

vectors = []
for d, e in zip(data, embeddings):
    vectors.append({
        "id": d['id'],
        "values": e['values'],
        "metadata": {'text': d['text']}
    })
index = pc.Index('llama-text-embed-v2-index')





In [89]:
import getpass
import os


os.environ["GROQ_API_KEY"] ="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"

In [90]:
pip install -qU langchain-groq

In [91]:
user_info={"user_id":123}

In [92]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [93]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
print(ai_msg.content)

J'adore la programmation.


In [94]:
from langchain.prompts import ChatPromptTemplate

controller_prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are a controller agent for a medical chatbot. Your job is to decide which of the following agents should handle the user query:

- Response_agent: Use this for general medical questions that do not require any personal user information. For example, if the user asks for general facts or opinions unrelated to their personal condition or history.

- retriever_agent: Use this when the user asks for medical advice or recommendations that depend on their personal data. If the user's own health, symptoms, or history are relevant, select this agent.

Reply with only one of the following:
- Response_agent
- retriever_agent
"""),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}")  # This is the correct role for tool responses or internal reasoning
])


In [95]:
from langchain.agents import create_tool_calling_agent
tools=[]

controller_agent = create_tool_calling_agent(llm, tools=tools,prompt=controller_prompt)

In [96]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.tools.pubmed.tool import PubmedQueryRun
from langchain.tools.render import render_text_description




pubmed= PubmedQueryRun()
rendered_tools = render_text_description([pubmed])

response_prompt = ChatPromptTemplate.from_messages([
    ("system", f"""
You are a response agent for a medical chatbot. Your job is to respond to user medical queries with reliable, evidence-based answers.

You have access to a tool {rendered_tools}. Use it when the user asks about:
- Conditions
- Diseases
- Treatments
- Drugs
- Medical research
- Symptoms
- Clinical guidelines

Only respond if the question is clearly medical. If the query is unrelated to health or medicine, reply: "I'm a medical chatbot and can only answer health-related questions."

Always use the PubMedSearch tool to gather relevant evidence when needed, before answering. However, filter the response to make it easy for the common person to understand.
Also only display the part of result that is relevant to the user query.Do not ask any follow up questions and just answer the user's query.
"""),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

In [97]:
!pip install langchain_community

In [98]:

response_agent = create_tool_calling_agent(llm, tools=[pubmed],prompt=response_prompt)

In [99]:
import uuid



def RAG_Agent(user_prompt):
  data={"id":user_info['user_id'],"text":user_prompt}
  x = pc.inference.embed(
    model="llama-text-embed-v2",
    inputs=[user_prompt],
    parameters={
        "input_type": "query"
    })

  vectors = []

  results = index.query(
    namespace="ns1",
    vector=x[0].values,
    top_k=3,
    include_values=False,
    include_metadata=True,
    filter={"user_id":123} )
  if results["matches"]:
    return results
  else:
    return "None"
def build_context(context,user_query):
  id1 = str(uuid.uuid4())
  id2 = str(uuid.uuid4())
  data=[{"id":id1,"text":user_query},{"id":id2,"text":context}]

  embeddings = pc.inference.embed(
    model="llama-text-embed-v2",
    inputs=[d['text'] for d in data],
    parameters={
        "input_type": "passage"
    }
)

  vectors = []
  for d, e in zip(data, embeddings):
      vectors.append({
          "id": d['id'],
          "values": e['values'],
          "metadata": {'text': d['text'],'user_id':user_info['user_id']}}
      )

  index.upsert(
      vectors=vectors,
      namespace="ns1"
  )






In [100]:
from langchain.prompts import ChatPromptTemplate

context_prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are a checker agent for a medical chatbot.

A Retrieval-Augmented Generation (RAG) system has fetched the top context documents for a given user query. If no context was fetched, then context will be False or None.

Your task is to analyze the user query and the retrieved context in **two steps**:

**Step 1: Context Relevance Check**
- Determine whether the retrieved context is relevant to the user query.
- If the context is relevant (i.e., it helps address or clarify the user’s query), proceed to Step 2 using the given context.
- If the context is **not** relevant to the user query, pretend that **no context was provided** and perform Step 2 as if `context = None`.

**Step 2: Sufficiency and Information Request**
- Based on the relevant context (or lack thereof), decide whether the available information is sufficient to answer the user query accurately.

**Output Format:**
- If sufficient, respond with:
  **Sufficient**
- If not sufficient, respond in a natural, conversational tone. Ask the user to provide the specific missing information needed to answer their query clearly and helpfully. Your response should flow like a chatbot message, such as:
  _"Thanks for your question! To help you better, could you please tell me more about...?"_
Note: Do not make comments about context as if you are an LLM infront of the user. The user should feel like he is interacting with a person
Use the following input format:
user: <user_query>
context: <retrieved_context> or None if no context
"""),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])


In [101]:
context_agent = create_tool_calling_agent(llm, tools=tools,prompt=context_prompt)

In [102]:
from langchain.agents import AgentExecutor

control_agent_executor = AgentExecutor(agent=controller_agent, tools=tools, verbose=True)

response_agent_executor = AgentExecutor(agent=response_agent, tools=[pubmed], verbose=True)

context_agent_executor = AgentExecutor(agent=context_agent, tools=tools, verbose=True)

In [103]:
user_query="I am having a severe headache"
selected_agent=control_agent_executor.invoke({"input":user_query})
selected_agent



> Entering new AgentExecutor chain...
retriever_agent

> Finished chain.


{'input': 'I am having a severe headache', 'output': 'retriever_agent'}

In [104]:
print(selected_agent)

{'input': 'I am having a severe headache', 'output': 'retriever_agent'}


In [105]:
if selected_agent['output'] == "Response_agent":
    result = response_agent_executor.invoke({"input": user_query})
else:
  results=RAG_Agent(user_query)

  formatted_query=''
  if results!="None":
    context_texts = [match['metadata']['text'] for match in results['matches']]
    combined_context = "\n".join(context_texts)
    formatted_query = f"user: {user_query}\ncontext: {combined_context}"
  else:
    formatted_query = f"user: {user_query}\ncontext:None"

  contexts=context_agent_executor.invoke({"input":formatted_query})
  context=''
  if contexts['output']!='Sufficient':

    context=input()
    build_context(context,user_query)
  final_query=user_query+formatted_query+context
  result = response_agent_executor.invoke({"input":final_query})
print(result['output'])








> Entering new AgentExecutor chain...
Thanks for sharing that with me. To help you better, could you please tell me more about any other symptoms you're experiencing, such as fever, nausea, or sensitivity to light, and also what you've tried so far to relieve the headache?

> Finished chain.
NO I do not have any other symtoms. I am just really sleepy


> Entering new AgentExecutor chain...

Invoking: `pub_med` with `{'query': 'severe headache causes'}`


Too Many Requests, waiting for 0.20 seconds...
Published: 2025-04-30
Title: Long-Term Cardiovascular Risk and Maternal History of Pre-Eclampsia.
Copyright Information: 
Summary::
Pre-eclampsia is a severe pregnancy complication affecting 5-8% of pregnancies worldwide, marked by high blood pressure and organ damage typically occurring after 20 weeks of gestation. It is a leading cause of maternal and fetal morbidity and mortality. Though its exact cause is unknown, it involves placental abnormalities and improper blood vessel developm

In [106]:
pubmed.invoke("What is panadol used for?")

Too Many Requests, waiting for 0.40 seconds...


'Published: 2025-05-14\nTitle: Quantitative Systems Toxicology Modeling of Acetaminophen Pharmacokinetics and Hepatic Biomarkers After Overdoses of Extended-Release and Immediate-Release Formulations in Adults With Chronic Alcohol Use or Low Glutathione.\nCopyright Information: © 2025 The Author(s). CPT: Pharmacometrics & Systems Pharmacology published by Wiley Periodicals LLC on behalf of American Society for Clinical Pharmacology and Therapeutics.\nSummary::\nAcetaminophen (APAP), an over-the-counter analgesic and antipyretic, can cause hepatotoxicity when ingested in large overdoses. APAP has multiple formulations including immediate-release (IR) and extended-release (ER) preparations. A recently published consensus statement on the management of APAP poisoning indicated that management of APAP-ER overdose is the same as that for APAP-IR overdose. Consistent with this consensus, it was previously reported that quantitative systems toxicology (QST) modeling using DILIsym predicted si